# Create Point Of Sales (POS)


This module create a POS device with its antenna and link it to a fixtures

Module interaction:

- Environement selection
- Premise selection
- Floor selection
- Area selection
- Zone selection
- Device type selection
- => device creation
- - ID : identifier of the device (string), must be unique in the organization. Auto generated by default
- - IP (mandatory) : ip address of the device on the local network, must be unique in the premise (unique in the organization for ShopCX)
- - NAME (mandatory) : friendly name of the device
- - MAC : MAC address of the device
- => POV auto creation and link to device 
- Fixture selection
- => fixture link to pov 


## Select an environment

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import settingsUtils as settingsUtils

import json
import ipaddress

In [ ]:
%%capture
status = True
myEnv = chooseEnv.choosePremise();
if not myEnv.get('urlDP'):
    text = 'No Environment selected '
    uiUtils.displayMessage('POS creation',text)
    print(text)
    status = False

In [ ]:
if status:
    print("Organization: " + myEnv.get('urlDP'))

## Create device

In [ ]:
%%capture
if status:
    myEnv = chooseEnv.chooseDeviceModel(myEnv, 'POS');
    deviceModel = myEnv['deviceModel'];
    if not deviceModel:
        text = 'No POS Model Found '
        uiUtils.displayMessage('POS creation',text)
        print(text)
        status = False
    else:
        print("Device Model ID: " + deviceModel['id'])

In [ ]:
%%capture
if status:
    myForm = uiUtils.inputWithForm('POS definition',['IP*','NAME*','MAC'])

    did = '' # myForm.get('ID') # NUMBER
    ip = myForm.get('IP*')
    ip = ip.strip()
    name = myForm.get('NAME*')
    mac = myForm.get('MAC')
    # check entries       
    if not name:
        text = 'POS Name is a mandatory fied '
        uiUtils.displayMessage('POS creation',text)
        print(text)
        status = False
        
    try:    
        tmp = ipaddress.ip_address(ip)
    except:
        text = 'POS IP address is not correct '
        uiUtils.displayMessage('POS creation',text)
        print(text)
        status = False        

In [ ]:
#%% CHECK ID UNICITY: deprecated
checkIpUnicity = True

In [ ]:
#%% CHECK IP UNICITY in the Premise
# DEV NOTE : should be integrated in the backEnd
if status and checkIpUnicity:
    res = deviceUtils.checkIPUnicityOnThePremise(myEnv,ip)
    if not res.get('status'):
        text = res.get('message')
        text  = text + '\n'+ ' POS can not been created'
        uiUtils.displayMessage('POS create',text)
        status = False

In [ ]:
if status: 
    deviceId = deviceUtils.createPOS(myEnv, ip, name, did, mac);
    if not deviceId:
        text = 'POS Creation: Error during creation of the device '
        uiUtils.displayMessage('POS creation',text)
        print(text)
        status = False

In [ ]:
if status:
    print("POS ID: " + deviceId)

## Create POV

In [ ]:
%%capture
if status:
    device = deviceUtils.getDeviceById(myEnv, deviceId);
    if not device:
        text = 'POS Creation: impossible to retrieve the device '
        uiUtils.displayMessage('POS creation',text)
        print(text)
        status = False

In [ ]:
if status:
    povId = povUtils.createPov(myEnv, '1', device['device']['name'] + '_1', deviceId + '_1', 'KEONN__ADPY__1')
    if not povId:
        text = 'POS Creation: impossible to create the PointOfView '
        uiUtils.displayMessage('POS creation',text)
        print(text)
        status = False

In [ ]:
if status:
    try :
        povUtils.linkPovToDevice(myEnv, povId, deviceId)
    except:
        text = 'POS Creation: impossible to link PointOfView to device '
        uiUtils.displayMessage('POS creation',text)
        print(text)
        status = False       

## Link to fixture

In [ ]:
%%capture
if status:
    myEnv = chooseEnv.chooseZone(myEnv);

In [ ]:
%%capture
if status:
    myEnv = chooseEnv.chooseFixture(myEnv, dialogTitle='Select fixture for pov ' + povId)
    fixtureId = myEnv['FixtureId']
    if not fixtureId:
        text = 'POS Creation: no valide fixture selected '
        uiUtils.displayMessage('POS creation',text)
        print(text)
        status = False  

In [ ]:
if status:
    try :
        povUtils.linkPovToFixture(myEnv, povId, fixtureId)
    except:
        text = 'POS Creation: impossible to link PointOfView to device '
        uiUtils.displayMessage('POS creation',text)
        print(text)
        status = False  

## Validate device has been created with default settings and start mode

## Get POS Device

In [ ]:
%%capture
if status:
    result = deviceUtils.getDeviceById(myEnv, deviceId)

In [ ]:
# display result
if status:
    text = json.dumps(result, indent=4, sort_keys=True)
    uiUtils.displayMessage('POS creation',text)
    print(text)

## Get RFID settings

In [ ]:
if status:
    settingsUtils.getRfidSettingsByDeviceId(myEnv, deviceId, 'POS_PAYMENT')

In [ ]:
if status:
    settingsUtils.getRfidSettingsByDeviceId(myEnv, deviceId, 'POS_RETURN')

In [ ]:
if status:
    settingsUtils.getRfidSettingsByDeviceId(myEnv, deviceId, 'POS_READ_ONLY')

## Get linked POV & mode

In [ ]:
%%capture
if status:
    settings = {}
    for pov in result['devicePov']:
        povId = pov['id']
        settings[povId + '_' + 'POS_PAYMENT'] = settingsUtils.getRfidSettingsByPovId(myEnv, povId,'POS_PAYMENT')
        settings[povId + '_' + 'POS_RETURN'] = settingsUtils.getRfidSettingsByPovId(myEnv, povId,'POS_RETURN')
        settings[povId + '_' + 'POS_READ_ONLY'] = settingsUtils.getRfidSettingsByPovId(myEnv, povId,'POS_READ_ONLY')

In [ ]:
if status:
    text = json.dumps(settings, indent=4, sort_keys=True)
    uiUtils.displayMessage('POS creation',text)
    print(text)